## Clean and Create Core Dataframes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
# List of countries to less the European Union
countries = ['GBR','USA','JPN','CHN','RUS','DEU','BRA','ARG','AUS','IDN','ITA','MEX']
deselected = ['United States','China']

In [3]:
def read_and_clean(csv_file):
    """
    Reader and does initial cleaning.

    Parameters
    ----------
    csv_file : file to be.

    Returns
    --------
    Dataframe with the desired years and countries.
    """
    
    file = pd.read_csv(csv_file, skiprows = 4).drop(['Indicator Name', 'Indicator Code', 'Unnamed: 65'], axis = 1)
    file = file[['Country Name','Country Code','2011','2012','2013','2014','2015','2016','2017','2018','2019']]
    cleaned = file[file['Country Code'].isin(countries)].sort_values(by = 'Country Name').drop('Country Code', axis = 1)
    cleaned['Country Name'] = cleaned['Country Name'].replace({'Russian Federation':'Russia','Korea, Rep.':'South Korea'})
    
    return cleaned

In [4]:
def scale_by_million(indicator_usd):
    """
    Scales the data by a million
    
    Parameters
    ----------
    indicator_usd: the economic indicator that needs to be scaled.
    
    Returns
    -------
    The dataframe with values scaled.
    
    """
    
    scaled = indicator_usd.set_index('Country Name').astype(float)/1000000
    return scaled.reset_index()

In [5]:
def redefine_df(df):
    """
    Redfines the dataframe, adding appropriate column labels.
    
    Parameter
    ---------
    df: the dataframe to which to add headers.
    """
    df.columns = ['2011','2012','2013','2014','2015','2016','2017','2018','2019']
    df.insert(loc = 0, column = 'Country Name', value = gdp_scaled['Country Name'])
    

In [6]:
# Read and clean all economic indicators to be used
edu_per_gdp = read_and_clean('educ_spending_per_gdp.csv')
health_per_gdp = read_and_clean('health_exp_per_gdp.csv')
health_per_cap = read_and_clean('health_exp_per_cap.csv')
military_per_gdp = read_and_clean('military_exp_per_gdp.csv')
military_usd = read_and_clean('military_exp_usd.csv')
gdp = read_and_clean('gdp_all.csv')
population = read_and_clean('population_all.csv')

In [7]:
# Scale the data
military_scaled = scale_by_million(military_usd)
pop_scaled = scale_by_million(population)
gdp_scaled = gdp.set_index('Country Name').astype(float)/1000000000  # Scale by Billion
gdp_scaled = gdp_scaled.reset_index()

In [8]:
"""
Create needed dataframes on per capita or fixed dollar amounts.
Uses the redefine method to formally create a new dataframe.
"""

# Create dataframe of military expenditure per capita 
military_per_cap = pd.DataFrame(military_usd.set_index('Country Name').values / population.set_index('Country Name').values)
redefine_df(military_per_cap)

# Create dataframe of healthcare expenditure in USD
health_usd = pd.DataFrame(health_per_cap.set_index('Country Name').values * population.set_index('Country Name').values)
redefine_df(health_usd)

# Create dataframe of education spending in USD
edu_usd = pd.DataFrame((edu_per_gdp.set_index('Country Name').values / 100) * gdp.set_index('Country Name').values)
redefine_df(edu_usd)

# Create dataframe of education spending per capita
edu_per_cap = pd.DataFrame(edu_usd.set_index('Country Name').values / population.set_index('Country Name').values)
redefine_df(edu_per_cap)

In [9]:
# Scaled the generated fixed value dataframes
edu_scaled = scale_by_million(edu_usd)
health_scaled = scale_by_million(health_usd)

In [10]:
def edu_imputer(df):
    """
    Imputed missing values for the passed in dataframe -- specifically related to education. 
    Selecting only the 10 countries with the most available data, this method generates 
    values using the average, respective to each country.
    
    Parameter
    ---------
    df: the education related dataframe to which to fill missing values using the average.
    
    Returns
    -------
    The generated dataframe with the imputed data for the 11 selected G20 countries (Argentina, 
    Australia, Brazil, Germany, Indonesia, Italy, Japan, Mexico, Russia, South Africa, United Kingdom).
    """
    # Select the 10 countries from the G20 that have the most available education spending data
    df_ten = df.loc[~df['Country Name'].isin(deselected)].drop(['2018','2019'],axis = 1).T
    
    # Create imputer to impute missing data based on the mean
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    imputer = imputer.fit(df_ten[1:])
    
    # Transform the data, imputing missing values and creating a dataframe
    df_imputed = pd.DataFrame(imputer.transform(df_ten[1:]))
    
    # Add appropriate labels
    df_imputed.columns = df[~(df['Country Name'].isin(deselected))]['Country Name'].to_list()
    df_imputed.insert(loc = 0, column = 'Country Name', value = ['2011','2012','2013','2014','2015','2016','2017'])

    
    # Add China back
    china_us = df.set_index('Country Name').T[deselected]
    df_imputed['China']= china_us['China']
    
    # Transpose the dataframe
    df_all_imputed = df_imputed.set_index('Country Name').T.reset_index().rename(columns = {'index':'Country'})
    df_all_imputed = df_all_imputed.sort_values(by = 'Country')
    
    return df_all_imputed

In [11]:
# # Impute missing values
edu_imputed = edu_imputer(edu_scaled)
edu_per_cap_imputed = edu_imputer(edu_per_cap)
edu_per_gdp_imputed = edu_imputer(edu_per_gdp)

In [12]:
# Create dataframe of healthcare expenditure in USD
gdp_per_cap = pd.DataFrame(gdp.set_index('Country Name').values / population.set_index('Country Name').values)
redefine_df(gdp_per_cap)

### US Education Exp

#### Scale to million, attach to edu_scaled (scaled 1m)

In [13]:
# Read in CSV of US education spending
df_us = pd.read_csv('US_education_exp.csv').rename(columns = {'G160291A027NBEA':'Education'})

# Scale to a million
us_millions = ((df_us.set_index('DATE').astype(float)*1000000000)/1000000)

# Cut to 2011 to 2017
us_millions_11_17 = us_millions[-9:-2:]

# Pivot table
us_pivoted = pd.pivot_table(us_millions_11_17, columns = 'DATE')

# Rename the columns to prep for concatination
us = us_pivoted.rename(columns = {'2011-01-01':'2011','2012-01-01':'2012','2013-01-01':'2013','2014-01-01':'2014'
                            ,'2015-01-01':'2015','2016-01-01':'2016','2017-01-01':'2017'})

# # Append US education data to the dataframe with all country education spending and drop previous US row
edu_imputed = edu_imputed.set_index('Country').append(us).reset_index().rename(columns = {'index':'Country'})

# # Replace row with US name and round the dataframe to two decimal places
edu_imputed = edu_imputed.replace('Education','United States').round(2)

#### Scale back to raw, get per capita and attach to edu_per_cap

In [14]:
# Scale back out of a original billion
us_b = df_us.set_index('DATE').astype(float)*1000000000

# Cut to 2011 to 2017
us_b_11_17 = us_b[-9:-2:]

# Pivot table
us_pivoted = pd.pivot_table(us_b_11_17, columns = 'DATE')

# Cut population years to match education years and get population values for the US
popneeded_years = population[population.loc[:,'Country Name'] == 'United States'].drop(['2018','2019'], axis = 1)
pop_values = popneeded_years.set_index('Country Name').values

# Get US education spending per cap
us_percap = pd.DataFrame((us_pivoted.values / pop_values))

# Relabel the columns as years
us_percap.columns = ['2011','2012','2013','2014','2015','2016','2017']

# Append the US education per cap data to the dataframe of other countries
edu_per_cap_imputed = edu_per_cap_imputed.append(us_percap).round(2)
edu_per_cap_imputed['Country'] = edu_per_cap_imputed['Country'].replace(np.nan,'United States')

#### Scale back to raw, get per gdp and attach to edu_per_gdp

In [15]:
# Scale back out of a original billion
us_b = df_us.set_index('DATE').astype(float)*1000000000

# Cut to 2011 to 2017
us_b_11_17 = us_b[-9:-2:]

# Pivot table
us_pivoted = pd.pivot_table(us_b_11_17, columns = 'DATE')

# Cut population years to match education years and get population values for the US
gdpneeded_years = gdp[gdp.loc[:,'Country Name'] == 'United States'].drop(['2018','2019'], axis = 1)
gdp_values = gdpneeded_years.set_index('Country Name').values

# Get US education spending per GDP
us_pergdp = pd.DataFrame((us_pivoted.values / gdp_values) * 100)

# Relabel the columns as years
us_pergdp.columns = ['2011','2012','2013','2014','2015','2016','2017']

# Append the US education per cap data to the dataframe of other countries
edu_per_gdp_imputed = edu_per_gdp_imputed.append(us_pergdp).round(2)
edu_per_gdp_imputed['Country'] = edu_per_gdp_imputed['Country'].replace(np.nan,'United States')

### Generate sheets for Correlations

In [16]:
# Create healthmilgdp_percap
healthmilgdp_percap = pd.DataFrame(list(zip(health_per_cap['Country Name'],
                                                  health_per_cap['2017'])), columns = ['Country','Healthcare'])

healthmilgdp_percap['Military'] = military_per_cap['2017']
healthmilgdp_percap['GDP'] = gdp_per_cap['2017']
healthmilgdp_percap = healthmilgdp_percap.round(2)
healthmilgdp_percap

,Country,Healthcare,Military,GDP
0,Argentina,1529.62,123.96,14613.04
1,Australia,5308.78,1125.57,54027.97
2,Brazil,935.30,140.90,9925.39
3,China,437.26,164.79,8879.44
4,Germany,5052.67,512.55,44552.82
5,Indonesia,110.05,33.24,3837.65
6,Italy,2809.72,436.89,32406.72
7,Japan,4121.04,357.98,38386.51
8,Mexico,512.07,40.57,9287.85
9,Russia,579.67,460.41,10894.36


In [17]:
# Create edumilgdp_percap
edumilgdp_percap = pd.DataFrame(list(zip(edu_per_cap_imputed['Country'],
                                                  edu_per_cap_imputed['2017'])), columns = ['Country','Education'])
edumilgdp_percap['Military'] = military_per_cap['2017']
edumilgdp_percap['GDP'] = gdp_per_cap['2017']
edumilgdp_percap = edumilgdp_percap.round(2)
edumilgdp_percap

,Country,Education,Military,GDP
0,Argentina,798.20,123.96,14613.04
1,Australia,2768.53,1125.57,54027.97
2,Brazil,627.54,140.90,9925.39
3,China,NaN,164.79,8879.44
4,Germany,2185.37,512.55,44552.82
5,Indonesia,119.60,33.24,3837.65
6,Italy,1310.83,436.89,32406.72
7,Japan,1221.53,357.98,38386.51
8,Mexico,420.07,40.57,9287.85
9,Russia,510.94,460.41,10894.36


In [18]:
# Create health_gdp_percap
health_gdp_percap = pd.DataFrame(list(zip(health_per_cap['Country Name'],
                                                  health_per_cap['2018'])), columns = ['Country','Healthcare'])
health_gdp_percap['GDP'] = gdp_per_cap['2018']
health_gdp_percap

,Country,Healthcare,GDP
0,Argentina,1127.907227,11633.498009
1,Australia,5425.340332,57354.964046
2,Brazil,848.388855,9001.234249
3,China,501.059387,9976.677137
4,Germany,5472.202148,47810.507671
5,Indonesia,111.677910,3893.846425
6,Italy,2988.995850,34615.756891
7,Japan,4266.586914,39159.423563
8,Mexico,519.605469,9686.513783
9,Russia,609.009155,11555.978814


In [19]:
# Create edu_gdp_percap
edu_gdp_percap = pd.DataFrame(list(zip(edu_per_cap_imputed['Country'],
                                                  edu_per_cap_imputed['2017'])), columns = ['Country','Education'])
edu_gdp_percap['GDP'] = gdp_per_cap['2017']
edu_gdp_percap = edu_gdp_percap.round(2)
edu_gdp_percap

,Country,Education,GDP
0,Argentina,798.20,14613.04
1,Australia,2768.53,54027.97
2,Brazil,627.54,9925.39
3,China,NaN,8879.44
4,Germany,2185.37,44552.82
5,Indonesia,119.60,3837.65
6,Italy,1310.83,32406.72
7,Japan,1221.53,38386.51
8,Mexico,420.07,9287.85
9,Russia,510.94,10894.36


In [20]:
# Create mil_gdp_percap
mil_gdp_percap = pd.DataFrame(list(zip(military_per_cap['Country Name'],
                                                  military_per_cap['2017'])), columns = ['Country','Military'])
mil_gdp_percap['GDP'] = gdp_per_cap['2017']
mil_gdp_percap = mil_gdp_percap.round(2)
mil_gdp_percap

,Country,Military,GDP
0,Argentina,123.96,14613.04
1,Australia,1125.57,54027.97
2,Brazil,140.90,9925.39
3,China,164.79,8879.44
4,Germany,512.55,44552.82
5,Indonesia,33.24,3837.65
6,Italy,436.89,32406.72
7,Japan,357.98,38386.51
8,Mexico,40.57,9287.85
9,Russia,460.41,10894.36


## Generate Excel Worksheet

In [23]:
"""
Create an excel workbook with the dataframes as separate sheets
"""

with pd.ExcelWriter('Project1_Cleaned.xlsx') as writer:
    gdp_scaled.to_excel(writer, sheet_name = 'gdp', index = False)
    pop_scaled.to_excel(writer, sheet_name = 'population', index = False)
    gdp_per_cap.to_excel(writer, sheet_name = 'gdp_per_cap', index = False)
    
    health_scaled.to_excel(writer, sheet_name = 'healthcare_exp', index = False)
    health_per_gdp.to_excel(writer, sheet_name = 'health_per_gdp', index = False)
    health_per_cap.to_excel(writer, sheet_name = 'health_per_cap', index = False)
    health_gdp_percap.to_excel(writer, sheet_name = 'health_gdp_percap', index = False)
    
    edu_imputed.to_excel(writer, sheet_name = 'edu_exp', index = False)
    edu_per_gdp_imputed.to_excel(writer, sheet_name = 'edu_per_gdp', index = False)
    edu_per_cap_imputed.to_excel(writer, sheet_name = 'edu_per_cap', index = False)
    edu_gdp_percap.to_excel(writer, sheet_name = 'edu_gdp_percap', index = False)
    
    military_scaled.to_excel(writer, sheet_name = 'military_exp', index = False)
    military_per_gdp.to_excel(writer, sheet_name = 'military_per_gdp', index = False)
    military_per_cap.to_excel(writer, sheet_name = 'military_per_cap', index = False)
    mil_gdp_percap.to_excel(writer, sheet_name = 'mil_gdp_percap', index = False)
    
    edumilgdp_percap.to_excel(writer, sheet_name = 'edumilgdp_percap', index = False)
    healthmilgdp_percap.to_excel(writer, sheet_name = 'healthmilgdp_percap', index = False)

### China Expenditure on education as percent of GDP (UNICEF)
https://www.unicef.cn/en/figure-826-government-expenditure-education-and-its-percentage-gdp-19922017


### Education Exp by country
https://nces.ed.gov/programs/coe/indicator_cmd.asp